In [6]:
import pandas as pd
import numpy as np
import sys

In [7]:
data = pd.read_csv('datasets/L-vs-R-news_labeled.csv')

In [8]:
data.columns

Index(['Unnamed: 0', 'right', 'left', 'sim', 'label'], dtype='object')

### Traditional Approach

In [9]:
import torch
from sentence_transformers import SentenceTransformer

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model_sent = SentenceTransformer('paraphrase-MiniLM-L6-v2')
model_sent.to(device)

/home/ahaque2/venv/py3_7/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [10]:
embedding_r = model_sent.encode(data.right)
embedding_r.shape

(3811, 384)

In [11]:
embedding_l = model_sent.encode(data.left)
embedding_l.shape

(3811, 384)

In [12]:
embeddings = np.concatenate((embedding_r, embedding_l), axis = 1)

In [13]:
embeddings.shape

(3811, 768)

In [18]:
from sklearn.model_selection import train_test_split
y = data['label']
X_train, X_test, y_train, y_test = train_test_split(embeddings, y,
                                                    stratify=y, 
                                                    test_size=0.25)

In [19]:
#Import svm model
from sklearn import svm

#Create a svm Classifier
clf = svm.SVC(kernel='linear') # Linear Kernel

#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

In [20]:
from sklearn import metrics

print("Accuracy:", round(metrics.accuracy_score(y_test, y_pred), 4))
print("Precision:", round(metrics.precision_score(y_test, y_pred), 4))
print("Recall:", round(metrics.recall_score(y_test, y_pred), 4))

Accuracy: 0.8269
Precision: 0.8193
Recall: 0.8382
